In [ ]:
import duckdb
import matplotlib.pyplot as plt
import polars as pl
import seaborn as sns

from config.seaborn import MONOKAI_OCTAGON, apply_style

# Set up connection and test query
conn = duckdb.connect("../../data/ff_platform.duckdb")

# Apply Seaborn Style
apply_style()

# Goal
In this workbook, my main goal is to asses some benchmarks around what metrics determine a "starter" in the NFL - with specific focus on Fantasy Football relevant positions. So `QB`, `RB`, `WR`, & `TE`. 

## Method & Process
Essentially, I am assuming that that `snap_counts` is the best signal on whether someone is a starter or not. Since I'm mostly interested in understanding position wide trends, I can somewhat ignore `injuries` - as if it a starter is injured, a back up will certainly take their place. 

I'm totall open to this assumption being wrong. But let's start here before over optimizing


In [ ]:
# Establish the query
query = """
select * 
from core.fct_player_game_stats 
"""
# Run Query
player_game_stats = conn.execute(query).pl()

# grab relevant players
target_stats = player_game_stats.filter(
    (pl.col('offense_snaps') >= 0.01) &
    (pl.col('position').is_in(['QB', 'RB', 'WR', 'TE']))
)

# show preview
target_stats.show()

In [ ]:
# let's quanitfy the data I'm working with - date ranges first
season_min = target_stats['season'].min()
season_max = target_stats['season'].max()
min_week = target_stats.filter((pl.col('season')==season_min))['week'].min()
max_week = target_stats.filter((pl.col('season')==season_max))['week'].max()
# share the date range
print(f"Dataset Range: { season_min } Week { min_week } - { season_max } Week { max_week } ")

In [ ]:
# aggregates by position across all seasons
aggregates_by_position = target_stats.group_by('position').agg(
    min=(pl.col('offense_pct').min()), 
    median=(pl.col('offense_pct').median()), 
    mean=pl.mean('offense_pct'), 
    max=pl.max('offense_pct')
    ).sort(by='position')
# print the aggregates
aggregates_by_position

In [ ]:
# let's check out the distribution of each position
TARGET_POSITION = 'WR'
player_dist = sns.displot(
        target_stats.filter(pl.col('position')==TARGET_POSITION).group_by('player_id').agg(pl.mean('offense_pct')), 
            x='offense_pct',
            )

# axis labels
player_dist.set_axis_labels(
                x_var="perc", 
                y_var="players"
            )
# title - for some reason this isn't setting, but it's fine for now
player_dist.set_titles(
                f"{TARGET_POSITION} Offensive Snap Count Perc"
            )
# display chart
player_dist


# Process Update
Alright, this is marginically helpful. I've got a quick way to see distrubtion of snaps across ALL players across all games. But really, I think I want to target an ordered rank of the top X by position. Let's breakdown the rules first...

### Position Rules
In the NFL, there is a hard 11 personnel cap on offense. 7 on the line of scrimmage & no more than 4 behind the line. 

We know there is: 
- 1 QB
- 5 O-Linemen

The **remaining 5 players** is where things feel questionable. Just some basic assumptions: 
1. WRs *generally* will never be more than 3
2. TEs will never be more than 2
3. RBs will never be more than 2

Assuming that all of those 3 positions will always have at least 1 player for that position - then we have 12 total combinations of positions. I'm not sure how that's relevant *now* 😅 but it might be later. 

So let's change this slightly then. I'm going to order the top `N` for each position by game by season. I still don't care about the actual individuals - just what the typical offensive start position was for: 
- QBs (kind of a moot point but - top 1)
- RBs (top 2 per team)
- WRs (top 3 per team)
- TEs (top 2 per team)





In [ ]:
# alright - so for each team for each season, I want to order the top X by position
def return_top_X_snaps(df, position, n_size):
    """
    Args: 
        - df: the snap counts df
        - position: a str, 'QB', 'RB', 'WR', 'TE' 
        - n_size: int, the top X size of players
    
    Output: 
        a df object that is just for the position specified n-size, to be used for visual analysis
    """
    # get the by game top 3 for the position
    by_game = df.filter(
      pl.col('position')==position
        ).with_columns(
      pl.col('offense_pct')
      .rank(method='ordinal', descending=True)
      .over(['game_id', 'team', 'season'])
      .alias('rank')
        ).filter(pl.col('rank') <= n_size)
    
     # After creating by_game, calculate the actual values:
    mean_val = by_game['offense_pct'].mean()
    median_val = by_game['offense_pct'].median()
    p25 = by_game['offense_pct'].quantile(0.25)
    p75 = by_game['offense_pct'].quantile(0.75)

    # Then create the chart
    chart = sns.displot(by_game, x='offense_pct')

    # Now use the computed values (not pl.col expressions)
    plt.axvline(mean_val, color=MONOKAI_OCTAGON[0], linestyle='--', label='Mean')
    plt.axvline(median_val, color=MONOKAI_OCTAGON[1], linestyle='--', label='Median')
    plt.axvline(p25, color=MONOKAI_OCTAGON[3], linestyle=':', label='25%-ile')
    plt.axvline(p75, color=MONOKAI_OCTAGON[3], linestyle=':', label='75%-ile')
    plt.xticks([0, 0.25, 0.5, 0.75, 1.0])
    plt.legend()

    ### Share found stats
    print(f"""
          Boundary Stats for { position }s - Top { n_size } per Game: 
          - 25%: { p25 }
          - mean: { mean_val }
          - median: { median_val }
          - 75%: { p75 }
          """)
    
    return chart

return_top_X_snaps(target_stats, 'TE', 2)

# Findings
While I won't document the exact findings here, I can at least say this logic is sound and would serve as a good baseline for other analysis down the line. I am going to tweak this slightly, focus on position players only (WR, RB, TE). 

Follow the logic at the `dbt/analysis/position_starter_stats.sql` model.